In [424]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import make_column_transformer

In [425]:
restaurant_data = pd.read_csv("/Users/chanwookim/Desktop/WebDev_Critics/GTWebDev-Critics/backend/data.csv")
restaurant_data.head()

,id,alias,name,image_url,is_closed,url,review_count,categories/0/alias,categories/0/title,categories/1/alias,...,location/city,location/zip_code,location/country,location/state,location/display_address/0,location/display_address/1,phone,display_phone,distance,location/display_address/2
0,U-i6cq-yFRVJC4pIKSLX9Q,poor-calvins-atlanta-15,Poor Calvin's,https://s3-media3.fl.yelpcdn.com/bphoto/OqNT3u...,False,https://www.yelp.com/biz/poor-calvins-atlanta-...,4351,asianfusion,Asian Fusion,southern,...,Atlanta,30308,US,GA,510 Piedmont Ave NE,"Atlanta, GA 30308",14042544051,(404) 254-4051,960.179637,NaN
1,GJxFtnTqTiokFedNrW9iDQ,atlanta-breakfast-club-atlanta,Atlanta Breakfast Club,https://s3-media3.fl.yelpcdn.com/bphoto/xBI3go...,False,https://www.yelp.com/biz/atlanta-breakfast-clu...,6148,southern,Southern,breakfast_brunch,...,Atlanta,30313,US,GA,249 Ivan Allen Jr Blvd,"Atlanta, GA 30313",14704283825,(470) 428-3825,2234.174969,NaN
2,z9oCxeqOKp2QMfM24m7bUw,mary-macs-tea-room-atlanta,Mary Mac's Tea Room,https://s3-media2.fl.yelpcdn.com/bphoto/1TSRr5...,False,https://www.yelp.com/biz/mary-macs-tea-room-at...,4737,southern,Southern,venues,...,Atlanta,30308,US,GA,224 Ponce De Leon Ave NE,"Atlanta, GA 30308",14048761800,(404) 876-1800,694.832218,NaN
3,BSUDAiXd50PAkfFAztVpDw,aviva-by-kameel-atlanta-atlanta,Aviva by Kameel - Atlanta,https://s3-media1.fl.yelpcdn.com/bphoto/5P3eyD...,False,https://www.yelp.com/biz/aviva-by-kameel-atlan...,1736,mediterranean,Mediterranean,mideastern,...,Atlanta,30303,US,GA,225 Peachtree St NE,Ste B-30,14046983600,(404) 698-3600,1789.863284,"Atlanta, GA 30303"
4,eG-UO83g_5zDk70FIJbm2w,south-city-kitchen-midtown-atlanta-2,South City Kitchen Midtown,https://s3-media3.fl.yelpcdn.com/bphoto/L1qX2t...,False,https://www.yelp.com/biz/south-city-kitchen-mi...,3087,southern,Southern,NaN,...,Atlanta,30309,US,GA,1144 Crescent Ave NE,"Atlanta, GA 30309",14048737358,(404) 873-7358,1925.444319,NaN


In [426]:
#dropping useless columns
restaurant = restaurant_data.drop(['url', 'id', 'alias', 'is_closed', 'categories/0/alias', 'phone', 'coordinates/latitude', 'coordinates/longitude', 'categories/1/alias', 'location/country', 'location/state', 'phone', 'image_url',
                                  'categories/1/title', 'categories/2/alias', 'transactions/1', 'categories/2/title', 'transactions/1', 'location/address2', 'location/address3', 'location/display_address/2', 'location/display_address/0',
                                  'location/display_address/1'], axis=1)
restaurant.duplicated().sum()
restaurant.drop_duplicates(inplace=True)


#Remove the NaN values from the dataset 
restaurant.isnull().sum()

#changing the column names 
restaurant = restaurant.rename(columns={'categories/0/title':'categories', 'location/city':'city', 'location/zip_code':'zip_code', 
                                      'location/country':'country', 'display_phone':'phone', 'location/address1':'address', 'transactions/0':'pick_up_or_delivery'})

In [427]:
restaurant = restaurant.sort_values(by='distance', na_position='first')
restaurant = restaurant.reset_index(drop=True)
restaurant = restaurant.drop([17])
restaurant = restaurant.reset_index(drop=True)

print(restaurant)

df_percent = restaurant.sample(frac=1.0)

## it holds any data type supported in Python and uses labels to locate each data value
indices = pd.Series(df_percent.index)

tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2))
tfidf.fit(df_percent['categories'])

tfidf_matrix = tfidf.fit_transform(df_percent['categories'])
tfidf_matrix.shape

from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tfidf_matrix)
cosine_sim_df = pd.DataFrame(cosine_sim, index=df_percent['name'], columns=df_percent['name'])
print("Shape:", cosine_sim_df.shape)
cosine_sim_df.sample(14, axis=1).round(2)

                                     name  review_count          categories  \
0                               Botiwalla           553              Indian   
1                     Mary Mac's Tea Room          4737            Southern   
2          Papi's Cuban & Caribbean Grill          1147               Cuban   
3                           Poor Calvin's          4351        Asian Fusion   
4                         Two Urban Licks          3295                Bars   
5              Herban Fix - Vegan Kitchen           902               Vegan   
6                         Highland Bakery          1781            Bakeries   
7           Flying Biscuit Café - Midtown          1875  Breakfast & Brunch   
8      The Vortex Bar And Grill - Midtown          1958             Burgers   
9             Cypress Street Pint & Plate          1473          Gastropubs   
10                           Ecco Midtown           972     Modern European   
11                   Barcelona Inman Park          1

name,Fox Bros. Bar-B-Q,Botiwalla,Barcelona Inman Park,Mary Mac's Tea Room,South City Kitchen Midtown,Aviva by Kameel - Atlanta,Highland Bakery,Hattie B's Hot Chicken - Atlanta East,Papi's Cuban & Caribbean Grill,Two Urban Licks,Ecco Midtown,Delbar Middle Eastern,The Vortex Bar And Grill - Midtown,Poor Calvin's
name,,,,,,,,,,,,,,
Flying Biscuit Café - Midtown,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Two Urban Licks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Fox Bros. Bar-B-Q,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Atlanta Breakfast Club,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
South City Kitchen Midtown,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cypress Street Pint & Plate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ecco Midtown,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
Herban Fix - Vegan Kitchen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Aviva by Kameel - Atlanta,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [428]:
def recommendations(category, M, items, k=10):
    # Filter items by category
    filtered_items = items[items['categories'] == category]
    return filtered_items

   

recommendations("Modern European", cosine_sim_df, df_percent[['name', 'categories']])




,name,categories
10,Ecco Midtown,Modern European
